In [8]:
import pandas as pd
import statsmodels.api as sm

# Download data
df = pd.read_csv("https://johanhombert.github.io/data/brk.csv")

# Select columns
df = df.iloc[:, :7]

print(df)

          month   BRK_ret      rf   mktrf     smb     hml     mom
0    01/01/1980  0.045455  0.0080  0.0551  0.0165  0.0172  0.0751
1    01/02/1980  0.000000  0.0089 -0.0122 -0.0183  0.0066  0.0788
2    01/03/1980 -0.217391  0.0121 -0.1290 -0.0665 -0.0102 -0.0959
3    01/04/1980  0.074074  0.0126  0.0397  0.0096  0.0101 -0.0042
4    01/05/1980  0.137931  0.0081  0.0526  0.0216  0.0038 -0.0111
..          ...       ...     ...     ...     ...     ...     ...
487  01/08/2020  0.115550  0.0001  0.0763 -0.0025 -0.0294  0.0051
488  01/09/2020 -0.023077  0.0001 -0.0363  0.0006 -0.0251  0.0305
489  01/10/2020 -0.054690  0.0001 -0.0210  0.0444  0.0403 -0.0303
490  01/11/2020  0.136159  0.0001  0.1247  0.0548  0.0211 -0.1225
491  01/12/2020  0.012008  0.0001  0.0463  0.0481 -0.0136 -0.0242

[492 rows x 7 columns]


In [9]:
# a. CAPM

# Dependent variable: Berkeshire Hathaway's excess return
df['BRK_retrf'] = df['BRK_ret'] - df['rf']
y = df['BRK_retrf']

# Regressors: market excess return and a constant 
X = df['mktrf']
X = sm.add_constant(X)

# Run OLS with robust standard errors (HC1)
model1 = sm.OLS(y, X).fit(cov_type='HC1')
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:              BRK_retrf   R-squared:                       0.237
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     131.2
Date:                Wed, 08 Oct 2025   Prob (F-statistic):           4.52e-27
Time:                        15:37:42   Log-Likelihood:                 719.42
No. Observations:                 492   AIC:                            -1435.
Df Residuals:                     490   BIC:                            -1426.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0079      0.003      3.108      0.0

In [10]:
# b. Fama-French 3-factor model

# Regressors
X = df[['mktrf','hml','smb']]
X = sm.add_constant(X)

# Run OLS
model2 = sm.OLS(y, X).fit(cov_type='HC1')
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:              BRK_retrf   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.307
Method:                 Least Squares   F-statistic:                     57.29
Date:                Wed, 08 Oct 2025   Prob (F-statistic):           9.60e-32
Time:                        15:37:42   Log-Likelihood:                 744.77
No. Observations:                 492   AIC:                            -1482.
Df Residuals:                     488   BIC:                            -1465.
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0068      0.002      2.780      0.0

In [11]:
# c. Expected return decomposition
# Contribution of exposure to market return
print("Market factor")
# = market risk premium ...
print("  risk premium:", df['mktrf'].mean())
# times BRK's exposure to market factor
print("  BRK's beta  :", model2.params['mktrf'])
# = ...
print("  contribution to ER:", df['mktrf'].mean() * model2.params['mktrf'])

# Same calculation for HML
print("HML factor")
print("  risk premium:", df['hml'].mean())
print("  BRK's beta  :", model2.params['hml'])
print("  contribution to ER:", df['hml'].mean() * model2.params['hml'])

# Same calculation for SMB
print("SMB factor")
print("  risk premium:", df['smb'].mean())
print("  BRK's beta  :", model2.params['smb'])
print("  contribution to ER:", df['smb'].mean() * model2.params['smb'])

Market factor
  risk premium: 0.007131504065040651
  BRK's beta  : 0.8127681805355077
  contribution to ER: 0.005796259583424667
HML factor
  risk premium: 0.0015949186991869918
  BRK's beta  : 0.4120389248184454
  contribution to ER: 0.0006571685859858416
SMB factor
  risk premium: 0.0009532520325203251
  BRK's beta  : -0.36972202269949095
  contribution to ER: -0.00035243826960581554


In [12]:
# Average excess return of BRK
df['BRK_retrf'].mean()

np.float64(0.012872223577235773)

In [13]:
# Average excess return of BRK minus contribution of each factor ==> is equal to estimated alpha
df['BRK_retrf'].mean() - df['mktrf'].mean() * model2.params['mktrf'] - df['hml'].mean() * model2.params['hml'] - df['smb'].mean() * model2.params['smb']

np.float64(0.00677123367743108)